In [ ]:
#########################
# Expiriments
#
# Loads in all the tumor images and runs the expiriments on the  CNNs
#

import pickle
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import (Conv2D, MaxPool2D, Flatten, Dense, Dropout, SpatialDropout2D)


PREDICTIONS_FILE = 'C:/Users/William/Desktop/CS 415 Project/WillKopec.pk1'


###########################
# Simple CNN Model
def create_model_Simple_CNN(num_classes, input_shape):
  model_Channel_Stacked = Sequential()
  model_Channel_Stacked.add(Conv2D(6, kernel_size = (5, 5), activation='relu', padding='same',
                input_shape=input_shape, kernel_initializer='random_normal'))
  model_Channel_Stacked.add(MaxPool2D((2,2)))

  model_Channel_Stacked.add(Conv2D(3, kernel_size = (10, 10), activation='relu', 
                padding='same', kernel_initializer='random_normal'))

  model_Channel_Stacked.add(Flatten())
  model_Channel_Stacked.add(Dense(100, activation='relu', kernel_initializer='random_normal'))
  model_Channel_Stacked.add(Dropout(0.5))
  model_Channel_Stacked.add(Dense(num_classes, activation='sigmoid', kernel_initializer='random_normal'))

  model_Channel_Stacked.compile(tf.keras.optimizers.Adam(0.0001), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model_Channel_Stacked

def run_trial(X, Y, model, prediction_file):
  from sklearn.model_selection import StratifiedShuffleSplit
  from keras.callbacks import EarlyStopping

  results = {}
  model.save_weights('checkpoints/my_checkpoint')

  sss = StratifiedShuffleSplit(n_splits=3, random_state=0)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

  indices = []
  predictions = []
  labels = []

  # Runs 3 iterations of a 5 fold cross validation
  for iteration in range(3):
    for train_index, test_index in sss.split(X, Y): # gets the train and test indecies
      model.load_weights('./checkpoints/my_checkpoint')

      train_index.astype(int, copy = False)
      test_index.astype(int, copy = False)

      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = Y[train_index], Y[test_index]
  
      model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 1000, batch_size = 50, callbacks = [es], verbose = 0)
      fold_predictions = model.predict(X_test)

      # saves the predictions
      indices += X_test.tolist()
      predictions += fold_predictions.tolist()
      labels += y_test.tolist()

  results['indices'] = indices
  results['predictions'] = predictions
  results['labels'] = labels

  open(prediction_file, 'wb').close()
  outfile = open(prediction_file, 'wb')


def create_dataset(path):
  dataset_dictionary = pickle.load(open(path, 'rb'))

  X, Y = [], []

  X_0 = np.array([x for x in dataset_dictionary['1'] if x.shape == (512, 512)])
  #X_1 = np.array([x for x in dataset_dictionary['2'] if x.shape == (512, 512)])
  #X_2 = np.array([x for x in dataset_dictionary['3'] if x.shape == (512, 512)])

  Y += [0] * X_0.shape[0]
  #Y += [1] * X_1.shape[0]
  #Y += [2] * X_2.shape[0]

  X = np.vstack([X_0])
  Y = np.array(Y)

  return X, Y


if __name__ == '__main__':
  ########################
  # Runs the trial with the simple crop features
  X, Y = create_dataset('C:/Users/William/Desktop/CS 415 Project/Preprocessed Data/Vanilla Crop/WillKopec.pk1')
  X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))
  print(X.shape)
  model = create_model_Simple_CNN(num_classes = 3, input_shape = (X.shape[1], X.shape[2], 1))
  run_trial(X, Y, model, PREDICTIONS_FILE)
  print(X.shape)


(296, 512, 512, 1)
